In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib

In [3]:
import string

In [17]:
r = urllib.urlopen('http://games.espn.com/ffl/recentactivity?leagueId=527677&seasonId=2017&activityType=-1&startDate=20160730&endDate=20171130&teamId=-1&tranType=-2').read()
soup = BeautifulSoup(r)
print type(soup)

<class 'bs4.BeautifulSoup'>


In [18]:
sss_franchises = np.array(['ROBO', 'HAGS', 'CHOD', 'FLY', 'BOWS', 'JAMM', 'CROW', 'CHUK', 'DEAD', 'YUMY', 'WHMA', 'PLOW'])

month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

In [19]:
rows = soup.find_all("tr")
cells = soup.find_all("td")

In [20]:
rows[5].get_text()

u'Wed, Nov 294:33 AMTransaction\xa0\xa0Add/DropDEAD added Ricky Seals-Jones, Ari TE from Waivers to BenchDEAD dropped Luke Willson, Sea TE to WaiversDEAD RosterWaiver Report'

In [21]:
print cells[6].get_text()
print cells[7].get_text()
print cells[8].get_text()
print cells[9].get_text()
print cells[10].get_text()
print cells[20].get_text()
print cells[75].get_text()
print cells[76].get_text()
print cells[150].get_text()
print cells[151].get_text()
print cells[152].get_text()
print cells[139].get_text()
print cells[140].get_text()

Wed, Nov 2912:30 PM
Transaction  Add/Drop
DEAD added Travaris Cadet, Buf RB from Free Agency to BenchDEAD dropped Damien Williams, Mia RB to Waivers
DEAD Roster
Wed, Nov 294:33 AM
ROBO dropped Jacoby Brissett, Ind QB to WaiversROBO added Trevor Siemian, Den QB from Waivers to Bench
Transaction  Add
CHOD added Adam Vinatieri, Ind K from Free Agency to Bench
Tue, Nov 141:46 PM
Transaction  Drop
ROBO dropped Redskins D/ST, Wsh D/ST to Waivers
Transaction  Add/Drop
WHMA dropped Rob Kelley*, Wsh RB to WaiversWHMA added Jacquizz Rodgers, TB RB from Waivers to Bench


In [113]:
print [x.get_text() for x in cells]

[u'\n\n\n\n\t\t\t\t// load new edition settings\n\t\t\t\tvar espn = espn || {};\n\t\t\t\tespn.i18n = espn.i18n || {};\n\t\t\t\tespn.i18n.lang = "en";\n\t\t\t\tespn.i18n.siteId = "1";\n\t\t\t\tespn.i18n.site = "espn";\n\t\t\t\tespn.i18n.editionKey = "espn-en";\n\t\t\t\tespn.i18n.personalization = true;\n\t\t\t\tespn.i18n.country = "us";\n\t\t\t\tespn.i18n.domain = "www.espn.com";\n\t\t\t\tespn.i18n.searchUrl = "http://search.espn.com/";\n\t\t\t\tespn.i18n.hasSearch = true;\n\t\t\t\tespn.i18n.nowFeed = true;\n\t\t\t\tespn.i18n.temperature = {scale: "fahrenheit", symbol: "&deg; F" };\n\t\t\t\tespn.i18n.facebook = {appId: "116656161708917", locale: "en_US" };\n\t\t\t\tespn.i18n.outbrain = {"mobile":{"compliantId":"MB_4","nonCompliantId":"MB_5"},"desktop":{"compliantId":"AR_15","nonCompliantId":"AR_12"},"video":{"mobile":{"compliantId":"MB_6"},"desktop":{"compliantId":"AR_16"}},"recap":{"mobile":{"compliantId":"MB_9"},"desktop":{"compliantId":"AR_30"}}}\n\t\t\t\tespn.i18n.betting = {show: t

In [22]:
loc = rows[4].get_text().find('Transaction')
print rows[4].get_text()[loc:loc+20]

Transaction  Add/Dro


In [194]:
#trade sequence:

#1. find franchises involved
#2. substitute the strings for pipe delimited
#3. split on the pip
#4. process each individually

def parse_transaction(line):
    
    transaction_types = []
    franchises = []
    players = []
    
    # Parse out multiple transactions
    if line.find('traded') >= 0:
        # hack at the stupid way ESPN splits up trade txns
        trading_franchises = list(sss_franchises[np.array([x in line for x in sss_franchises])])
        combinations = [trading_franchises[0]+trading_franchises[0],trading_franchises[0]+trading_franchises[1], trading_franchises[1]+trading_franchises[0], trading_franchises[1]+trading_franchises[1]]
        replacements = [trading_franchises[0]+'|'+trading_franchises[0],trading_franchises[0]+'|'+trading_franchises[1], trading_franchises[1]+'|'+trading_franchises[0], trading_franchises[1]+'|'+trading_franchises[1]]
        newline = line
        for i in range(len(combinations)):
            newline = newline.replace(combinations[i], replacements[i])
        transactions = newline.split('|')
    elif line.find('added') >= 0 and line.find('dropped') >= 0:
        if line.find('added') < line.find('dropped'):
            #print line
            transactions = line.split('Bench')
        else:
            if line.find('to Waivers') >= 0:
                transactions = line.split('Waivers')
            else:
                transactions = line.split('Free Agency')
    else:
        transactions = [line]
    
    # Deal with individual parsed transactions
    for transaction in transactions:
        if transaction.find('added') >= 0:
            # parse the details
            transaction_type = 'add'
            splits = transaction.split('added')
            franchise = splits[0][:-1]
            player = splits[1].split(',')[0][1:].replace("*",'')
            # add to the list
            franchises.append(franchise)
            players.append(player)
            transaction_types.append(transaction_type)
            
        elif transaction.find('dropped') >= 0:
            # parse the details
            transaction_type = 'drop'
            splits = transaction.split('dropped')
            franchise = splits[0][:-1]
            player = splits[1].split(',')[0][1:].replace("*",'')
            # add to the list
            franchises.append(franchise)
            players.append(player)
            transaction_types.append(transaction_type)
            
        elif transaction.find('traded') >= 0:
            
            # drop from the first team
            # parse the details
            transaction_type = 'drop'
            splits = transaction.split('traded')
            franchise = splits[0][:-1]
            player = splits[1].split(',')[0][1:].replace("*",'')
            # add to the list
            franchises.append(franchise)
            players.append(player)
            transaction_types.append(transaction_type)
            
            # add to the other one
            # parse the details
            transaction_type = 'add'
            splits = transaction.split('to')
            franchise = splits[1][1:]
            player = player
            # add to the list
            franchises.append(franchise)
            players.append(player)
            transaction_types.append(transaction_type)
            
    return([franchises, transaction_types, players])


In [ ]:
flow:
    
1. scrape
2. keep string for later
3. get parsed
4. database it

In [223]:
transactions = []
datetimes = []
franchises = []
players = []
transaction_types = []
i = 0
transaction_counter = 0

while i < len(cells):
    # start at the 6th cell
    if i < 6:
        i = 6
    
    # Need to get the type first - if veto, just do opposite of the txn - drop adds, add drops, etc.
    # aka, if it's a veto, the txn type is drop/add if add/drop, or whatever
    
    type_text = cells[i+1].get_text()
    transaction_text = cells[i+2].get_text()
    transactions.append(transaction_text)
    print type_text
    print transaction_text
    
    # get the datetime
    datetime_text = cells[i].get_text()
    datetimes.append(get_datetime(datetime_text))
    
    # parse the transaction itself
    # TODO: don't parse if it's a trade besides trade upheld
    result = parse_transaction(transaction_text)
    franchises.extend(result[0])
    transaction_types.extend(result[1])
    players.extend(result[2])
    
    print datetime_text
    print datetimes[len(datetimes)-1]
    print result[0]
    print result[1]
    print result[2]
    
    
    i += 4
    


Transaction  Add
HAGS added Alex Smith, KC QB from Free Agency to Bench
Fri, Sep 303:04 PM
2016-09-30 3:04:00
[u'HAGS']
['add']
[u'Alex Smith']
Transaction  Drop
HAGS dropped Greg Zuerlein, LA K to Waivers
Fri, Sep 303:04 PM
2016-09-30 3:04:00
[u'HAGS']
['drop']
[u'Greg Zuerlein']
Transaction  Add
HAGS added Dustin Hopkins, Wsh K from Free Agency to Bench
Fri, Sep 302:55 PM
2016-09-30 2:55:00
[u'HAGS']
['add']
[u'Dustin Hopkins']
Transaction  Drop
HAGS dropped Billy Cundiff, Cle K to Waivers
Fri, Sep 302:50 PM
2016-09-30 2:50:00
[u'HAGS']
['drop']
[u'Billy Cundiff']
Transaction  Add/Drop
JAMM dropped Seth Roberts, Oak WR to WaiversJAMM added Steelers D/ST, Pit D/ST from Free Agency to Bench
Thu, Sep 2912:13 PM
2016-09-29 12:13:00
[u'JAMM', u'JAMM']
['drop', 'add']
[u'Seth Roberts', u'Steelers D/ST']
Transaction  Drop
BOWS dropped Bennie Fowler, Den WR to Waivers
Thu, Sep 2912:54 AM
2016-09-29 12:54:00
[u'BOWS']
['drop']
[u'Bennie Fowler']
Transaction  Add
BOWS added Rishard Matthews, T

In [224]:
zip(datetimes,franchises, transaction_types, players)

[(u'2016-09-30 3:04:00', u'HAGS', 'add', u'Alex Smith'),
 (u'2016-09-30 3:04:00', u'HAGS', 'drop', u'Greg Zuerlein'),
 (u'2016-09-30 2:55:00', u'HAGS', 'add', u'Dustin Hopkins'),
 (u'2016-09-30 2:50:00', u'HAGS', 'drop', u'Billy Cundiff'),
 (u'2016-09-29 12:13:00', u'JAMM', 'drop', u'Seth Roberts'),
 (u'2016-09-29 12:54:00', u'JAMM', 'add', u'Steelers D/ST'),
 (u'2016-09-29 12:54:00', u'BOWS', 'drop', u'Bennie Fowler'),
 (u'2016-09-29 12:53:00', u'BOWS', 'add', u'Rishard Matthews'),
 (u'2016-09-29 12:16:00', u'BOWS', 'drop', u'Rishard Matthews'),
 (u'2016-09-28 10:01:00', u'BOWS', 'add', u'Rishard Matthews'),
 (u'2016-09-02 85:34:00', u'PLOW', 'drop', u'Chris Boswell'),
 (u'2016-09-02 85:33:00', u'PLOW', 'add', u'Blair Walsh'),
 (u'2016-09-28 11:52:00', u'BOWS', 'drop', u'Jared Cook'),
 (u'2016-09-28 11:19:00', u'BOWS', 'add', u'Raiders D/ST'),
 (u'2016-09-28 10:49:00', u'CHOD', 'drop', u'Mike Tolbert'),
 (u'2016-09-28 10:49:00', u'CHOD', 'drop', u'Virgil Green'),
 (u'2016-09-28 10:32:

In [174]:
line = full.iloc[30,1]
print line

ROBO dropped Dolphins D/ST, Mia D/ST to WaiversCHOD traded James White, NE RB to ROBO


In [175]:
line.split('Waivers')

[u'ROBO dropped Dolphins D/ST, Mia D/ST to ',
 u'CHOD traded James White, NE RB to ROBO']

In [162]:
parse_transaction(line)

[[u'JAMM dropped Seth Roberts, Oak WR to WaiversJAMM'],
 ['add'],
 [u'Steelers D/ST']]

In [19]:
full = pd.DataFrame(zip(transaction_types, transactions))
print full[full[0] == 'Trade Upheld (By LM)']
#full


                        0                                                  1
29   Trade Upheld (By LM)           ROBO traded Kyle Rudolph, Min TE to CHOD
40   Trade Upheld (By LM)  PLOW traded Russell Wilson, Sea QB to ROBOROBO...
42   Trade Upheld (By LM)  FLY traded Ryan Mathews, Phi RB to PLOWPLOW tr...
46   Trade Upheld (By LM)  PLOW traded Josh Gordon, Cle WR to FLYFLY drop...
57   Trade Upheld (By LM)           ROBO traded Charles Clay, Buf TE to HAGS
59   Trade Upheld (By LM)  DEAD traded Bengals D/ST, Cin D/ST to PLOWDEAD...
61   Trade Upheld (By LM)  HAGS traded Travis Kelce, KC TE to ROBOROBO tr...
90   Trade Upheld (By LM)  JAMM traded Justin Forsett, Bal RB to PLOWPLOW...
92   Trade Upheld (By LM)  JAMM traded Matthew Stafford, Det QB to ROBORO...
95   Trade Upheld (By LM)           DEAD traded Cody Latimer, Den WR to CHUK
158  Trade Upheld (By LM)  FLY traded Michael Crabtree, Oak WR to ROBOROB...


In [18]:
print transactions[40]

PLOW traded Russell Wilson, Sea QB to ROBOROBO traded Eric Ebron*, Det TE to PLOWROBO traded Matthew Stafford, Det QB to PLOWROBO traded Christine Michael, Sea RB to PLOWPLOW dropped Vance McDonald*, SF TE to Waivers


In [142]:
print full[full[0] == 'Drop (By LM)']

                0                                                  1
15   Drop (By LM)  JAMM dropped Steelers D/ST, Pit D/ST to Free A...
32   Drop (By LM)   CHOD dropped Rishard Matthews, Ten WR to Waivers
80   Drop (By LM)     HAGS dropped DeAndre Smelter, SF WR to Waivers
101  Drop (By LM)        HAGS dropped Tyler Varga, Ind RB to Waivers
102  Drop (By LM)      HAGS dropped Mario Alford*, Cin WR to Waivers
103  Drop (By LM)        HAGS dropped Sean Mannion, LA QB to Waivers
488  Drop (By LM)        CHOD dropped James Starks, GB RB to Waivers
489  Drop (By LM)     CHOD dropped Brandon McManus, Den K to Waivers
490  Drop (By LM)       CHOD dropped Jason Witten, Dal TE to Waivers
491  Drop (By LM)      CHOD dropped Alfred Morris, Dal RB to Waivers
492  Drop (By LM)       PLOW dropped Justin Hardy, Atl WR to Waivers
493  Drop (By LM)     PLOW dropped Delanie Walker, Ten TE to Waivers
494  Drop (By LM)        PLOW dropped Zach Zenner, Det RB to Waivers
495  Drop (By LM)          PLOW dr

In [182]:
full.iloc[496,1]

u'PLOW dropped Bills D/ST, Buf D/ST to Waivers'

In [180]:
full.iloc[496,1].split('dropped')[1].split(',')[0]

u' Bills D/ST'

In [161]:
print transactions[4]

JAMM dropped Seth Roberts, Oak WR to WaiversJAMM added Steelers D/ST, Pit D/ST from Free Agency to Bench


In [ ]:
Sep 311:35 AM
Aug 306:51 PM
Aug 3011:42 PM
Sep 285:34 PM
Sep 211:35 PM

In [ ]:
# Sep 211:35 PM is ambiguous - don't know if it's sep 2 or sep 21. 
#In this case, need to infer the date from surrounding transactions
# and work from there

In [227]:
daytime = '285:34 PM'
if daytime[1] == '0':
    day = daytime[0:2]
    hour = daytime[2]
    minute = daytime[4:6]
else:
    day = '0' + daytime[0]
    hour = daytime[1:3]
    minute = daytime[4:6]

In [228]:
print day
print hour
print minute

02
85
34


In [222]:
def get_franchise(text):
    loc = text.find(' ')
    franchise = text[:loc]
    return franchise

def get_player(text):
    loc = text.find(' ', 6)
    loc2 = text.find(',')
    player = text[loc+1:loc2]
    return player

def get_datetime(text):
    loc = text.find(' ')
    month = text[loc+1:loc+4]
    loc2 = text.find(' ', loc+5)
    daytime = text[loc+5:loc2]
    if len(daytime) == 5:
        day = '0' + daytime[0]
        hour = daytime[1]
        minute = daytime[3:5]
    elif len(daytime) == 6:
        if daytime[1] == '0':
            day = daytime[0:2]
            hour = daytime[2]
            minute = daytime[4:6]
        else:
            day = '0' + daytime[0]
            hour = daytime[1:3]
            minute = daytime[4:6]
    else:
        day = daytime[0:2]
        hour = daytime[2:4]
        minute = daytime[5:7]
        
    datetime = '2016-' + month_dict[month] + '-' + day + ' ' + hour + ':' + minute + ':' + '00'
    
    return datetime

def get_transaction_type(text):
    loc = text.find('Transaction')
    transaction_type = text[loc+13:]
    return transaction_type

In [ ]:
26, 29, 30, 31, 33, 36

In [136]:
types[26:39]

[u'Trade Processed',
 u'Add',
 u'Drop',
 u'Trade Upheld (By LM)',
 u'CHOD Accepted Trade',
 u'CHOD Accepted Trade',
 u'Drop (By LM)',
 u'Trade (Vetoed) (By LM)',
 u'Add',
 u'Drop',
 u'ROBO Accepted Trade',
 u'Add/Drop',
 u'Add']

In [137]:
players[26:39]

[u'Cameron Brate',
 u'dropped Brock Osweiler',
 u'Jimmy Garoppolo',
 u'Orleans Darkwa',
 u'Eagles D/ST',
 u'dropped Buccaneers D/ST',
 u'Buccaneers D/ST',
 u'Cowboys D/ST',
 u'Cowboys D/ST',
 u'Austin Seferian-Jenkins',
 u'Joe Flacco',
 u'dropped Brian Hoyer',
 u'Bills D/ST']

In [56]:
a = 'Waivers then'

In [60]:
a.find('ives')

-1

In [110]:
a = 'add'
b = 'drop'
c = 'drop'

c in [a,b]

True